In [1]:
from __future__ import print_function
from torchvision import models
import torch
import os
import torch.nn as nn
import numpy as np

model_weights_path = '/mnt/chart/output/vocal-sunset-38/weights'
test_img_dir = '/mnt/chart/raw/test_release/task1/png'
output_dir = '/mnt/chart/test_results'

In [9]:
a = np.load(model_weights_path, allow_pickle=True)

In [2]:
device = torch.device("cuda:0")

model = models.resnet50(pretrained=False)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 10)

if torch.cuda.device_count() > 1:
    print("Let's use", torch.cuda.device_count(), "GPUs!")
    model = nn.DataParallel(model)
    
model.load_state_dict(torch.load(model_weights_path))
model = model.to(device)
model.eval() 

Let's use 3 GPUs!


DataParallel(
  (module): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequential(
      

In [27]:
from torchvision import transforms
import skimage

transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

img_name = './samples/3dpie.png'
image = skimage.io.imread(img_name)[:,:,:3]

image_t = transform(image)
image_t = image_t.unsqueeze(0)
image_t = image_t.to(device)
image_t.shape

torch.Size([1, 3, 224, 224])

In [28]:
outputs = model(image_t)
outputs

tensor([[ -6.8840,   8.2656,   2.9011, -11.7606, -13.9996,  14.5862, -17.3097,
          11.9888, -10.3453, -16.4582]], device='cuda:0',
       grad_fn=<GatherBackward>)

In [29]:
_, predicted = torch.max(outputs.data, 1)

In [30]:
predicted

tensor([5], device='cuda:0')

In [18]:
from src.datapreprocessing import DataProprocessing
provider = DataProprocessing('/mnt/chart/raw/images', None, '/mnt/chart-classification/labels_task1.csv', seed=443)

In [21]:
train_dataset = provider.get_train_dataset()

In [24]:
train_dataset.codec.classes_

array(['Donut', 'Grouped horizontal bar', 'Grouped vertical bar',
       'Horizontal box', 'Line', 'Pie', 'Scatter',
       'Stacked horizontal bar', 'Stacked vertical bar', 'Vertical box'],
      dtype='<U22')

In [31]:
train_dataset.codec.inverse_transform([5])

array(['Pie'], dtype='<U22')

In [51]:
class TestDataset(torch.utils.data.Dataset):
    def __init__(self, images_dir, transform=None):        
        self.root_dir = images_dir
        self.transform = transform        
        self.images = [x for x in os.listdir(self.root_dir) if x[-3:] == 'png']
        
    def __len__(self):
        return len(self.images)
    
    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()       
        image = self.read_image(idx)
        if self.transform:
            image = self.transform(image)
        return (image, self.images[idx])
    
    def read_image(self, idx):
        img_id = self.images[idx]
        print(img_id)
        img_name = os.path.join(self.root_dir, img_id)
        image = skimage.io.imread(img_name)[:,:,:3]
        return image

In [52]:
test_path = '/mnt/chart/raw/test_release/task1/png'
output_path = '/mnt/chart/test_results'

test_dataset = TestDataset(test_path)

In [53]:
len(test_dataset)

1221

In [57]:
test_dataset[0][1][:-4]

1186.png


'1186'

In [59]:
import os
import torch
import skimage
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision import models
import torch.nn as nn

test_path = '/mnt/chart/raw/test_release/task1/png'
output_path = '/mnt/chart/test_results'
model_weights_path = '/mnt/chart/output/splendid-valley-40/weights'

classes = ['Donut', 'Grouped horizontal bar', 'Grouped vertical bar',
           'Horizontal box', 'Line', 'Pie', 'Scatter',
           'Stacked horizontal bar', 'Stacked vertical bar', 'Vertical box']

transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])
test_dataset = TestDataset(test_path, transform=transform)

loader = DataLoader(test_dataset, batch_size=128, num_workers=16)

# load model
device = torch.device("cuda:0")
model = models.resnet50(pretrained=False)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 10)

model.load_state_dict(torch.load(model_weights_path))
if torch.cuda.device_count() > 1:
    model = nn.DataParallel(model)

model = model.to(device)
model.eval() 

for images, names in loader:
    images.to(device)
    outputs = model(images)
    _, predicted = torch.max(outputs.data, 1)
    
    print(predicted)
    break

7318.png
7055.png
1186.png
100.png
1478.png
7428.png
5378.png
1300.png
6181.png
7528.png
7196.png
1192.png
6760.png
1430.png
1494.png
4874.png
1314.png
1159.png
7547.png
1188.png
1179.png
4822.png
5180.png
3010.png
3855.png
6335.png
5130.png
5781.png
4021.png
1136.png
5731.png
7508.png
1161.png
1328.png
2593.png
1480.png
1356.png
1281.png
6296.png
4974.png
6659.png
6411.png
1145.png
7164.png
5725.png
7510.png
114.png
1295.png
7553.png
1418.png
7534.png
4019.png
7602.png
2710.png
7163.png
7542.png
6805.png
1256.png
1122.png
3353.png
7252.png
1274.png
1219.png
1133.png
1198.png
7169.png
5313.png
5422.png
1273.png
5044.png
7605.png
4983.png
1345.png
5427.png
3957.png
6511.png
1175.png
1267.png
7584.png
1222.png
3463.png
1423.png
5341.png
4976.png
1083.png
7504.png
5445.png
1501.png
4366.png
7590.png
1437.png
5012.png
7556.png
4689.png
4962.png
6626.png
5451.png
7158.png
6798.png
3090.png
1351.png
1127.png
1260.png
1097.png
1149.png
4745.png
7246.png
1298.png
3838.png
7589.png
1379.png
753

7533.png
4994.png
1464.png
7117.png
7541.png
6710.png
7549.png
3149.png
1142.png
4758.png
1470.png
1130.png
6209.png
7392.png
5219.png
1038.png
1110.png
5324.png
1316.png
7569.png
3037.png
6701.png
1042.png
7103.png
7561.png
7527.png
1010.png
4285.png
1118.png
1146.png
5552.png
7433.png
5362.png
1156.png
1214.png
3069.png
6486.png
7537.png
1052.png
5504.png
3655.png
1104.png
1181.png
7559.png
6663.png
1393.png
6235.png
1056.png
7575.png
1004.png
5234.png
1128.png
7596.png
3590.png
3359.png
7341.png
5495.png
1448.png
1005.png
6729.png
6109.png
7582.png
1387.png
6962.png
6936.png
7282.png
1011.png
2633.png
1046.png
7217.png
7090.png
3832.png
7586.png
1081.png
8062.png
1312.png
1195.png
3160.png
7571.png
4599.png
6792.png
6698.png
1095.png
1039.png
1183.png
1047.png
1460.png
2665.png
7498.png
1100.png
4925.png
6756.png
6820.png
1053.png
6275.png
1306.png
1378.png
7315.png
5366.png
4800.png
7592.png
2572.png
1197.png
1268.png
1436.png
3717.png
3511.png
3809.png
1134.png
5440.png
1002.png
7

In [61]:
predicted

tensor([1, 2, 2, 6, 1, 6, 0, 4, 5, 5, 0, 5, 2, 6, 4, 5, 7, 1, 4, 0, 0, 1, 2, 2,
        2, 1, 4, 0, 5, 4, 0, 4, 4, 4, 5, 4, 1, 5, 0, 2, 0, 0, 0, 2, 5, 4, 1, 2,
        1, 4, 5, 1, 4, 4, 0, 5, 0, 4, 5, 8, 5, 2, 0, 1, 5, 8, 4, 0, 1, 7, 0, 5,
        0, 2, 4, 5, 6, 1, 6, 2, 6, 2, 2, 2, 5, 2, 5, 1, 5, 5, 2, 6, 4, 4, 4, 7,
        6, 7, 4, 2, 0, 4, 4, 1, 2, 5, 4, 7, 7, 2, 5, 5, 1, 5, 1, 5, 0, 0, 0, 4,
        0, 0, 0, 1, 0, 1, 2, 1], device='cuda:0')

In [64]:
labs = train_dataset.codec.inverse_transform(predicted.cpu())

In [65]:
for x in zip(names, labs):
    print(x)

('1186.png', 'Grouped horizontal bar')
('1192.png', 'Grouped vertical bar')
('1179.png', 'Grouped vertical bar')
('7508.png', 'Scatter')
('1145.png', 'Grouped horizontal bar')
('7534.png', 'Scatter')
('7252.png', 'Donut')
('4983.png', 'Line')
('5445.png', 'Pie')
('5451.png', 'Pie')
('7246.png', 'Donut')
('6158.png', 'Pie')
('1151.png', 'Grouped vertical bar')
('7520.png', 'Scatter')
('3746.png', 'Line')
('5337.png', 'Pie')
('1384.png', 'Stacked horizontal bar')
('1390.png', 'Grouped horizontal bar')
('3587.png', 'Line')
('7087.png', 'Donut')
('6372.png', 'Donut')
('1409.png', 'Grouped horizontal bar')
('1421.png', 'Grouped vertical bar')
('1347.png', 'Grouped vertical bar')
('1353.png', 'Grouped vertical bar')
('1435.png', 'Grouped horizontal bar')
('4377.png', 'Line')
('6548.png', 'Donut')
('5727.png', 'Pie')
('4439.png', 'Line')
('6979.png', 'Donut')
('4834.png', 'Line')
('3183.png', 'Line')
('4607.png', 'Line')
('5519.png', 'Pie')
('2510.png', 'Line')
('1019.png', 'Grouped horizonta

In [69]:
import json
def save_json_output(output, out_dir):
    name = output[0]
    label = output[1]
    pred = {
        "task1": {
            "output": {
                "chart_type": output[1]
            }
        }
    }
    with open(os.path.join(out_dir, name + ".json"), 'w') as f:
        json.dump(pred, f)

save_json_output((names[0], labs[0]), '/mnt/chart/')